In [ ]:
!pip install crewai pandas numpy

In [ ]:
!pip install dotenv

In [2]:
# --- Load environment variables ---
import os 
import pandas as pd 

from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not set in .env file")

# Set OpenAI API key for langchain or crewai usage
os.environ["OPENAI_API_KEY"] = openai_api_key

In [10]:
from crewai import Agent, Task, Crew, Process

In [4]:
## Load the dataset 
df = pd.read_csv('product_reviews.csv')
df.head()

,review_text
0,Affordable and works as expected.
1,Highly recommend to others.
2,Terrible experience. Would not buy again.
3,Doesn't match the description online.
4,Affordable and works as expected.


In [ ]:
## Agent 1 : Cleaner 

cleaner = Agent(name = 'Review Cleaner', 
                role= 'Cleaner',
                goal='Filter out short reviews <20 characters', 
                backstory='An expert in filtering out noise from customer feedback data',
                verbose=True)

task1 = Task(name = 'Clean Short Reviews',
             description='Remove rows where review_text is less than 20 characters',
             expected_output='List of reviews longer than 20 characters',
             agent = cleaner,
             input = df)

In [8]:
## Agent 2 : Summarizer 

summarizer = Agent(name = 'Review Summarizer', 
                role= 'Summarizer',
                goal='Identify common themes and sentiments in customer reviews', 
                backstory='Specializes in natural language summarization of customer opinions',
                verbose=True)

task2 = Task(name = 'Summarize Reviews',
             description='Identify the top 3 common themes and sentiments in customer reviews and give examples',
             expected_output='3 review themes with examples customer reviews',
             agent = summarizer)

In [11]:
## Create the crew and run the show 

crew = Crew(agents = [cleaner, summarizer],
            tasks = [task1, task2],
            process=Process.sequential, 
            verbose = True)

In [12]:
result = crew.kickoff()
print(result)

┌────────────────────────── Crew Execution Started ───────────────────────────┐
│                                                                             │
│  Crew Execution Started                                                     │
│  Name: crew                                                                 │
│  ID: 3237b9bf-dbad-4a60-aae4-072690430a1f                                   │
│  Tool Args:                                                                 │
│                                                                             │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘

🚀 Crew: crew
└── 📋 Task: f61f16a7-1424-4986-a1c2-d658dd5a0078
    Status: Executing Task...
┌───────────────────────────── 🤖 Agent Started ──────────────────────────────┐
│                                                                             │
│  Agent: Cleaner           

c:\Users\jaina\anaconda3\envs\agentic_AI\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='I now ca...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


CrewOutput(raw='**Theme 1: Product Quality and Performance**  \nMany customers express high satisfaction with the quality and performance of the products. For example, one review states, "This product works wonderfully and exceeded my expectations!" Another customer mentions, "The quality of the materials used in this product is top-notch!" indicating a strong perception of durability and effectiveness. Furthermore, a review highlights, "The features of this item make it worth every penny!" symbolizing that customers feel they receive great value for their money.\n\n**Theme 2: Positive Customer Experience**  \nCustomers frequently highlight their positive experiences with customer service and communication from the sellers. One reviewer shares, "I experienced great customer service with this company, and I will recommend it to others," showing appreciation for effective support. Another customer adds, "I will be buying from this seller again due to their excellent communication," which